## Install required libraries

<a href="https://colab.research.google.com/github/RedisVentures/redis-google-llms/blob/main/BigQuery_Palm_Redis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
!pip install redis "google-cloud-aiplatform==1.25.0" --upgrade --user

^^^ If prompted press the Restart button to restart the kernel. ^^^

In [50]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb focal main
Starting redis-stack-server, database path /var/lib/redis-stack


gpg: cannot open '/dev/tty': No such device or address
(23) Failed writing body


## Connect to Redis server

In [66]:
import os
import redis
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#Replace values above with your own if using Redis Cloud instance
#REDIS_HOST="redis-12110.c82.us-east-1-2.ec2.cloud.redislabs.com"
#REDIS_PORT=12110
#REDIS_PASSWORD="pobhBJP7Psicp2gV0iqa2ZOc1WdXXXXX"

#shortcut for redis-cli $REDIS_CONN command
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"
redis = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)
redis.ping()

True

## Authenticate to Google Cloud

In [52]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [53]:
project_id = 'central-beach-194106'

## Big Query SQL to Datafame

In [54]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

df = client.query('''
SELECT * FROM `bigquery-public-data.hacker_news.full`
where type !='comment' LIMIT 1000
''').to_dataframe()

display(df)

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,Ask HN: Why is engagement on Twitter so unbeli...,None,Example: Ycombinator has 1 million followers. ...,<NA>,TekMol,1,1541794502,2018-11-09 20:15:02+00:00,story,18417552,<NA>,6,<NA>,<NA>
1,Ask HN: Was the U.S. election hacked?,None,I haven&#x27;t seen any discussion on this at ...,<NA>,awatlits,1,1478788916,2016-11-10 14:41:56+00:00,story,12920705,<NA>,7,<NA>,<NA>
2,Ask HN: How often do you WFH?,None,Working as a software engineer in London I fin...,<NA>,_ao789,1,1498810013,2017-06-30 08:06:53+00:00,story,14669261,<NA>,6,<NA>,<NA>
3,"Ask HN: Engineers, how do you share code with ...",None,My team and I are all frustrated by the curren...,<NA>,aismail,1,1467301075,2016-06-30 15:37:55+00:00,story,12009280,<NA>,6,<NA>,<NA>
4,Off-topic: Why do people put [scribd] in a pos...,None,"Wouldn't putting ""[pdf]"" in the title be more ...",<NA>,ncarlson,1,1247864664,2009-07-17 21:04:24+00:00,story,710788,<NA>,12,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Computer Technical Support to Fix Windows 7 Di...,http://www.amazingarticles.com/computer-techni...,There can be varied problems on the background...,True,kellymarion6,1,1329729889,2012-02-20 09:24:49+00:00,story,3611848,<NA>,-1,<NA>,<NA>
996,Eco-friendly Sony Ericsson Xperia Neo V for so...,http://bit.ly/x8lsHi,Sony Ericsson Xperia Neo V is an eco-friendly ...,True,catjohnes,1,1328104291,2012-02-01 13:51:31+00:00,story,3537883,<NA>,-1,<NA>,<NA>
997,Lift Tables,http://www.pentalift.com/scissor-lift.htm,"Pentalift designs, engineers, and manufactures...",True,charlenellsm,1,1240793864,2009-04-27 00:57:44+00:00,story,580660,<NA>,-1,<NA>,<NA>
998,Summer camp,http://www.hiddenbayleadershipcamp.ca,"Ontario Summer Camp, Hidden Bay Leadership Cam...",True,charlenellsm,1,1240793878,2009-04-27 00:57:58+00:00,story,580662,<NA>,-1,<NA>,<NA>


## Redis Helper functions

In [80]:
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()

def load_dataframe(redis, df, key_prefix="tweet", id_column="id", pipe_size=100):
    records = df.to_dict(orient="records")
    pipe = redis.pipeline()
    i=1
    for record in tqdm(records):
        #print(record)
        # Convert <NA> values to an empty string, timestamp, bool to string representation
        # Nnot sure if it belons here or in df API, but the problem it addresses is specific to redis-py
        converted_record = {
            key: '' if pd.isna(value) else str(value) if isinstance(value, (pd.Timestamp, bool)) else value
            for key, value in record.items()
        }
        #print(converted_record)
        record=converted_record
        i=i+1
        key = f"{key_prefix}:{record[id_column]}"
        pipe.hset(key, mapping=record)
        if (i+1) % pipe_size == 0:
          res=pipe.execute()
    pipe.execute()

## Load Dataframe to Redis

In [81]:
# clear Redis database (optional)
redis.flushdb()

# load data from Dataframe to Redis HASH
load_dataframe(redis,df,key_prefix="google", id_column="id", pipe_size=100)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [83]:
#!redis-cli $REDIS_CONN KEYS "*"

In [86]:
#retreive single HASH from Redis
redis.hgetall(f"google:{df.loc[1, 'id']}")

{b'title': b'Ask HN: Was the U.S. election hacked?',
 b'url': b'',
 b'text': b'I haven&#x27;t seen any discussion on this at all.',
 b'dead': b'',
 b'by': b'awatlits',
 b'score': b'1',
 b'time': b'1478788916',
 b'timestamp': b'2016-11-10 14:41:56+00:00',
 b'type': b'story',
 b'id': b'12920705',
 b'parent': b'',
 b'descendants': b'7',
 b'ranking': b'',
 b'deleted': b''}

## Init Vertex AI

In [57]:
import vertexai

PROJECT_ID = project_id
vertexai.init(project=PROJECT_ID, location="us-central1")

In [58]:
from vertexai.preview.language_models import (ChatModel, InputOutputTextPair,
                                              TextEmbeddingModel,
                                              TextGenerationModel)

## Hello Palm!

In [59]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

prompt = "What is a large language model?"

response = generation_model.predict(prompt=prompt)

print(response.text)

A large language model (LLM) is a type of artificial intelligence (AI) model that can understand and generate human language. LLMs are trained on massive datasets of text and code, and they can learn to perform a wide variety of tasks, such as translating languages, writing different kinds of creative content, and answering your questions in an informative way.

LLMs are still under development, but they have the potential to revolutionize many industries. For example, LLMs could be used to create more accurate and personalized customer service experiences, to help doctors diagnose and treat diseases, and to even write entire books and movies.




## Hello Chat!

In [60]:

chat_model = ChatModel.from_pretrained("chat-bison@001")

chat = chat_model.start_chat()

print(
    chat.send_message(
        """
Hello! Can you write a 300 word abstract for a research paper I need to write about the impact of generative AI on society?
"""
    )
)


print(
    chat.send_message(
        """
Could you give me a catchy title for the paper?
"""
    )
)

Generative AI (GAN) is a type of machine learning that uses artificial neural networks to create new, original content. This can include text, images, audio, and video. GANs have the potential to revolutionize many industries, from healthcare to advertising. However, there are also concerns about the potential negative impacts of GANs, such as the creation of fake news and deepfakes.

In this paper, we explore the potential impact of GANs on society. We first discuss the benefits of GANs, such as their ability to create new and original content, to solve complex problems, and to personalize experiences. We then discuss
The Impact of Generative AI on Society
